In [1]:
import pandas as pd
import numpy as np
import requests
import json
import config
import io
import os
import dotenv
import sys
sys.tracebacklimit = 0

import geopandas as gpd
from geopandas import GeoDataFrame, points_from_xy
from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer, basemaps
from cartoframes.data.clients import SQLClient

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
credentials = pd.read_csv('credentials.csv',header=0)
username = credentials.columns[0]
api_key = credentials.values[0][0]

set_default_credentials(username=username,api_key=api_key)

credentials = Credentials(username=username,api_key=api_key)

sql = SQLClient(credentials)

In [3]:
zipper = pd.read_csv('Miami Zip codes.csv')
zips = zipper['Zip Code'].tolist()
zips.sort()

In [4]:
headers = {'Content-type': 'application/json'}

In [5]:
employ_dict = {
    'LAUDV123312400000005' : 'employment', ## Total Employment Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331240000000001' : 'tot_non_farm', ## Total Non-Farm Employ Numbers: Miami-Miami Beach-Kendall, FL(sum of tot_govt & tot_private)
    'SMU12331249000000001' : 'tot_govt', ## Total Government Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331249091000001' : 'fed_govt', ## Federal Government Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331249092000001' : 'state_govt', ## State Government Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331249093000001' : 'local_govt', ## Local Government Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331240500000001' : 'tot_private', ## Total private sector Employ Numbers: sum of goods prod & service provid
    'SMU12331240600000001' : 'goods_prod', ## Total goods producing Employ Numbers: sum of mine_log_constr & manuf
    'SMU12331241500000001' : 'mine_log_construct', ## Total mining, logging, construction Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331243000000001' : 'manufact', ## Total manufacturing Employ Numbers: sum of durable & non-durable goods
    'SMU12331240800000001' : 'private_serv_prov', ## Total private service providing Employ Numbers: sum of below 7 industries
    'SMU12331246500000001' : 'ed_health_serv', ## Total education & health services Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331247000000001' : 'leis_hospitality', ## Total leisure & hospitality Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331245500000001' : 'finance', ## Total financial activites Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331246000000001' : 'pro_biz_serv', ## Total professional business services employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331244000000001' : 'trade_trans_util', ## Total trade, transportation & utilities Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331245000000001' : 'info', ## Total information industry Employ Numbers: Miami-Miami Beach-Kendall, FL
    'SMU12331248000000001' : 'other', ## Total other service industries Employ Numbers: Miami-Miami Beach-Kendall, FL
}

In [6]:
employ_data1 = json.dumps({
    "seriesid": list(employ_dict.keys()),
    "startyear": '2003',
    "endyear": '2012'
})

employ_data2 = json.dumps({
    "seriesid": list(employ_dict.keys()),
    "startyear": '2013',
    "endyear": '2021'
})

In [7]:
employ_pull1 = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/',data=employ_data1,headers=headers).json()['Results']['series']
employ_pull2 = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/',data=employ_data2,headers=headers).json()['Results']['series']

In [8]:
date_list1 = [f"{i['year']}-{i['period'][1:]}-01" for i in employ_pull1[0]['data']]
# Empty dataframe to fill with values
employ1 = pd.DataFrame()
# Build a pandas series from the API results
for s in employ_pull1:
    employ1[employ_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list1),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
    
date_list2 = [f"{i['year']}-{i['period'][1:]}-01" for i in employ_pull2[0]['data']]
# Empty dataframe to fill with values
employ2 = pd.DataFrame()
# Build a pandas series from the API results
for s in employ_pull2:
    employ2[employ_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list2),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
    
## Join into 1 DF
employ = pd.concat([employ1, employ2], axis=0)
## Employment only variable not in thousands
## convert, so it matches the other variables
employ['employment'] = employ['employment']/1000
## Farm jobs not reported; can calc by just
## subtracting total_non_farm from employment
employ['tot_farm'] = employ['employment'] - employ['tot_non_farm']

In [9]:
employ

,employment,tot_non_farm,tot_govt,fed_govt,state_govt,local_govt,tot_private,goods_prod,mine_log_construct,manufact,private_serv_prov,ed_health_serv,leis_hospitality,finance,pro_biz_serv,trade_trans_util,info,other,tot_farm
2003-01-01,1034.145,985.7,151.8,20.5,17.7,113.6,833.9,94.1,40.4,53.7,739.8,127.9,93.1,69.1,124.2,254.4,25.4,45.7,48.445
2003-02-01,1020.362,993.5,152.7,20.5,18.6,113.6,840.8,94.2,40.5,53.7,746.6,129.6,95.0,69.4,127.5,253.7,25.1,46.3,26.862
2003-03-01,1004.581,997.9,153.6,20.6,18.9,114.1,844.3,94.5,41.1,53.4,749.8,130.4,96.5,69.4,128.6,253.3,25.1,46.5,6.681
2003-04-01,1019.543,990.1,153.8,20.6,18.8,114.4,836.3,93.1,41.1,52.0,743.2,130.3,94.4,69.0,125.2,253.3,24.9,46.1,29.443
2003-05-01,1030.977,989.7,155.2,20.5,19.1,115.6,834.5,93.1,41.4,51.7,741.4,130.2,94.1,69.1,124.0,252.8,24.8,46.4,41.277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,1228.229,1138.6,130.7,20.6,26.3,83.8,1007.9,91.2,50.6,40.6,916.7,182.6,116.6,83.0,187.5,280.8,18.3,47.9,89.629
2021-07-01,1277.593,1145.9,131.1,20.8,26.1,84.2,1014.8,90.6,50.1,40.5,924.2,183.6,117.4,85.3,188.0,284.0,18.3,47.6,131.693
2021-08-01,1268.014,1160.9,140.6,20.7,26.0,93.9,1020.3,90.3,50.4,39.9,930.0,184.2,117.5,85.4,190.5,286.4,18.5,47.5,107.114
2021-09-01,1282.942,1163.4,140.3,21.1,27.2,92.0,1023.1,92.6,51.5,41.1,930.5,185.5,116.3,84.4,191.9,286.2,18.4,47.8,119.542


In [10]:
## Below was to verify all industries add up to tot_non_farm
# df = employ.copy()
# df.drop(['employment','fed_govt','state_govt','local_govt','tot_private','goods_prod','private_serv_prov','tot_farm'], 
#         axis=1, inplace=True)
# df

In [11]:
## Export to csv
employ.to_csv("emp.csv", sep=",", index=False)

In [12]:
employr = employ.reset_index()
employr=employr.rename(columns={'index':'date'})

In [13]:
## Push to Carto
to_carto(employr, 'emp', if_exists='replace')

Success! Data uploaded to table "emp" correctly


'emp'